In [8]:
#python sudoku driver

# lines = [[],[],[],[],[],[],[],[],[]]
# board = np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]
#          ,[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
# with open('board.txt') as f:
#     for i in range(9):
#         lines[i] = f.readline().splitlines()
#         for j in range(9):
#             board[i][j] = int(lines[i][0][j])

import numpy as np

def make_board():
    with open("board.txt") as f:
        return np.array(list(map(lambda line: list(line.strip()), f.readlines()))).astype("int64")

board = make_board()

tiles = [0]*9
for i in range(3):
    tiles[i] = np.array((board[0][0 + i*3:3 + i*3],
                         board[1][0 + i*3:3 + i*3],
                         board[2][0 + i*3:3 + i*3]))
    tiles[i + 3] = np.array((board[3][0 + i*3:3 + i*3],
                             board[4][0 + i*3:3 + i*3],
                             board[5][0 + i*3:3 + i*3]))
    tiles[i + 6] = np.array((board[6][0 + i*3:3 + i*3],
                             board[7][0 + i*3:3 + i*3],
                             board[8][0 + i*3:3 + i*3]))

unknowns = []
for i in range(9):
    for j in range(9):
        if board[i][j] == 0:
            unknowns.append((i,j))

            
def board_to_tile_coords(m,n):
    tile = what_tile(m,n)
    if tile in range(3):
        row = m
    elif tile in range(3,6):
        row = m - 3
    else:
        row = m - 6
    if tile in (0,3,6):
        col = n
    elif tile in (1,4,7):
        col = n - 3
    else:
        col = n - 6
    return (row,col)
    

#modify the board array AND the tiles array
def update_board(m,n,updated_value):
    tile_coords = board_to_tile_coords(m,n)
    tiles[what_tile(m,n)][tile_coords[0]][tile_coords[1]] = updated_value
    board[m][n] = updated_value
    
    
#returns the index of a tile in the tiles array, given coordinates
def what_tile(m,n):
    a = 0
    for i in range(3):
        if n in range (0 + i*3,3 + i*3) and m in range(3):
            return i
        elif n in range(0 + i*3,3 + i*3) and m in range(3,6):
            return i + 3
        elif n in range(0 + i*3,3 + i*3) and m in range(6,9):
            return i + 6

#checks if the value in a square is legal
def check_space(m,n):   
    num = board[m][n]
    tile_in = what_tile(m,n)
    m_count = list(board[m]).count(num)
    n_count = list(board[:,n]).count(num)
    tile_count = 0
    for i in range(3):
        tile_count += list(tiles[tile_in][i]).count(num)
    if m_count > 1 or n_count > 1 or tile_count > 1:
        return False
    else:
        return True

#return the list of valid numbers to enter in an unknown square
#the current number (if nonzero) is considered valid
def avail_nums(m,n): 
    tile = what_tile(m,n)
    a = set(list(board[m]) + list(board[:,n]) + list(tiles[tile][0]) 
            + list(tiles[tile][1]) + list(tiles[tile][2]))
    b = {1,2,3,4,5,6,7,8,9}
    if board[m][n] != 0:
        c = sorted(b.difference(a)) + [board[m][n]]
    else:
        c = b.difference(a)
    return sorted(c)

#returns the next-largest legal number for a square. only call next_avail when there is such a number.
def next_avail(m,n,curr_val):
    avail = avail_nums(m,n)
    if curr_val == 0:
        return avail[0]
    elif curr_val != 0 and avail.index(curr_val) < len(avail) - 1:
        at = avail.index(curr_val)
        return avail[at + 1]

#returns boolean for whether the value in a given square is the greatest legal number
def is_max(m,n,curr_val):
    avail = avail_nums(m,n)
    if 0 <= avail.index(curr_val) < (len(avail) - 1):
        return False
    elif avail.index(curr_val) == len(avail) - 1:
        return True
    
def is_empty(thing):
    return not len(thing)
    
# def is_solved():
#     count = 0
#     ideal = [1,2,3,4,5,6,7,8,9]
#     for i in range(9):
#         if sorted(set(tiles[i][0] + tiles[i][1] + tiles[i][2])) == ideal  and sorted(set(board[i])) == ideal \
#                                                                     and sorted(set(board[:,i])) == ideal:
#             count += 1
#     if count == 9:
#         return True
#     else:
#         return False

In [9]:
#backtracking algo - brute force

from IPython.display import clear_output
import time
start = time.time()
j = 0# last-changed square counter
(m,n) = unknowns[j]
num_unknowns = len(unknowns)
run = 0
print_freq = 10000 #larger ==> less frequent

while True:
    if board[m][n] == 0 and is_empty(avail_nums(m,n)) == False: #forward - succeed
        update_board(m,n,next_avail(m,n,board[m][n]))
        j += 1
        if j < num_unknowns:
            (m,n) = unknowns[j]
        else:
            break
    elif board[m][n] == 0 and is_empty(avail_nums(m,n)) == True: # forward - fail
        j -= 1
        (m,n) = unknowns[j]
    elif is_max(m,n,board[m][n]) == False: #backward - succeed
        update_board(m,n,next_avail(m,n,board[m][n]))
        j += 1
        (m,n) = unknowns[j]
    else: #backward - fail
        update_board(m,n,0)
        if j > 0:
            j -= 1
        (m,n) = unknowns[j]
#     uncomment below to see the board update as it runs. 
    run += 1
    if run % print_freq == 0:
        clear_output(wait=True)
        print(board, run)    

            
print(board, run)
print("took only " + str(time.time() - start) + " secs")
print("😼 ")

[[1 5 2 8 4 7 6 9 3]
 [4 3 6 5 2 9 1 7 8]
 [7 8 9 6 3 1 5 2 4]
 [8 7 5 3 1 2 9 4 6]
 [9 1 4 0 8 0 0 0 2]
 [6 0 0 0 0 4 0 0 0]
 [3 0 0 0 0 0 0 1 0]
 [0 4 0 0 0 0 0 0 7]
 [0 0 7 0 0 0 3 0 0]] 10000
[[1 6 2 8 5 7 4 9 3]
 [5 3 4 1 2 9 6 7 8]
 [7 8 9 6 4 3 5 2 1]
 [4 7 5 3 1 2 9 8 6]
 [9 1 3 5 8 6 7 4 2]
 [6 2 8 7 9 4 1 3 5]
 [3 5 6 4 7 8 2 1 9]
 [2 4 1 9 3 5 8 6 7]
 [8 9 7 2 6 1 3 5 4]] 17879
took only 1.9183192253112793 secs
😼 
